In [ ]:
!mkdir -p data
!wget -nc https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1s/XRPUSDT-1s-2025-02.zip -O data/XRPUSDT-1s-2025-02.zip

In [ ]:
import zipfile

import numpy as np

import binance_column_names as bcn

with zipfile.ZipFile('data/XRPUSDT-1s-2025-02.zip', 'r') as zf:
	with zf.open('XRPUSDT-1s-2025-02.csv') as f:
		data = np.loadtxt(
			f,
			delimiter=',',
			usecols=range(0, 6), # open time, open, high, low, close, volume
			dtype={
				'names': bcn.names[:6],
				'formats': [np.int64] + [np.float64] * 5,
			},
			# skiprows=5*24*60*60,
			max_rows=17*24*60*60,
		)

In [ ]:
!cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -Bbuild && cmake --build build --target rnn_trader

import ctypes

lib =  ctypes.CDLL('./build/librnn_trader.so')
n_var = ctypes.c_size_t.in_dll(lib, 'rnn_trader_var').value

rnn_trader_run = lib.rnn_trader_run
rnn_trader_run.restype = None
rnn_trader_run.argtypes = [
	ctypes.c_void_p, # candles
	ctypes.c_size_t, # candle_count
	ctypes.c_double * n_var, # params
	ctypes.c_void_p, # out
]

In [ ]:
from IPython.display import display_markdown
import pathlib
from multiprocessing.pool import ThreadPool as Pool

pathlib.Path('params').mkdir(exist_ok=True)
pool = Pool()

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.evaluator import Evaluator
from pymoo.core.problem import Problem
from pymoo.core.termination import NoTermination
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.survival.rank_and_crowding import RankAndCrowding
from pymoo.problems.static import StaticProblem

POP_SIZE = 60
SUBPERIODS = 3
F = np.empty((POP_SIZE, SUBPERIODS + 1), dtype=np.float64) # return per subperiod, and total return
OUT = np.empty((POP_SIZE, SUBPERIODS), dtype=[('ret', 'f8'), ('mdd', 'f8'), ('npos', 'i8')])

problem = Problem(
	n_var=n_var,
	n_obj=SUBPERIODS + 1,
	n_constr=0,
	xl=-3,
	xu=+3,
)

algorithm = NSGA2(
	pop_size=POP_SIZE,
	crossover=UniformCrossover(prob=1.0),
	survival=RankAndCrowding(crowding_func='mnn'),
)
algorithm.setup(problem, termination=NoTermination());

In [ ]:
# rerun for as many generations as you need
pop = algorithm.ask()
if algorithm.n_gen == 1:
	try:
		for p, r in zip(pop, np.loadtxt(f'params/{n_var}.txt', delimiter=',', ndmin=2)): p.X = r
	except FileNotFoundError: pass

for subperiod in range(SUBPERIODS):
	candles_per_subperiod = data.shape[0] // SUBPERIODS
	candles = data[subperiod * candles_per_subperiod:].ctypes.data_as(ctypes.c_void_p)
	pool.starmap(
		lambda individual, out: rnn_trader_run(
			candles, candles_per_subperiod,
			np.ctypeslib.as_ctypes(individual.X), np.ctypeslib.as_ctypes(out)
		),
		zip(pop, OUT[:, subperiod].view((np.float64, 3))),
	)
	F[:, subperiod] = -OUT[:, subperiod]['ret'] # negating for maximization
F[:, -1] = F[:, :-1].sum(axis=1) # total return
Evaluator().eval(StaticProblem(problem, F=F, out=OUT), pop)
algorithm.tell(infills=pop)

res = algorithm.result()

In [ ]:
profits, tot_profits, out = -res.F[:, :-1], -res.F[:, -1], res.pop.get('out')

output = f'gen: {algorithm.n_gen - 1}  always-profitable: {(profits > 0).all(axis=1).sum()}\n'
output += '| profits | tot. profit |\n| --- | --: |\n'
for profit, tot_profit, npos in zip(profits, tot_profits, out['npos'].sum(axis=1)):
	if not ((npos and profit.min() > -1) or tot_profit > 0): continue

	output += f'| {np.array2string(profit, precision=4)[1:-1]} | {tot_profit:.4f} |\n'

display_markdown(output, raw=True)
with open(f'params/{n_var}.txt', 'w') as f: np.savetxt(f, res.X[(profits > 0).all(axis=1)], delimiter=', ')

gen: 1  always-profitable: 2
| profits | tot. profit |
| --- | --: |
| 0.1493 0.9784 1.5726 | 2.7003 |
| 0.0801 4.7306 1.5924 | 6.4030 |
